## Predznanje

- Namesto dveh značilk za temperaturo bom uvedel novo spremenljivko delta_t
- poleg spremenljivke theta bom uvedel še cos(theta)
- zdi se smiselno uvesti spremenljivko 1/izolacijski_indeks

Enote nam v tem primeru ne pomagajo, saj je količina, ki nosi neko informacijo o enotah le temperatura. Enote izolacijskega indeksa so verjetno take, da se v enačbi lepo pokrajšajo v W. Ker enote indeksa niso podane, si z njimi ne moremo pomagati. Očitno je le, da se je treba delta_t pomnožiti z neko drugo enačbo, da je Celzija izniči.

## Algoritem - naiven poskus
Pričakujem, da formula za toplotni tok posnema preproste oblike osnovnih formul pri toploti (Recimo Q=-kA deltaT). Zato bom poskusil z generiranjem vseh možnih enačb kratke oblike, ki posnemajo zgoraj izpeljana pravila. Formule bom generiral s pomočjo ProGeda, ker z gramatikami na enostaven način opišem željen prostor enačb.



In [1]:
import ProGED as pg
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# priprava podatkov
data = pd.read_csv('DN4_1_podatki.csv')
data['deltaT'] = data['Tw'] - data['Ta']
data['one_over_eta'] = 1 / data['eta']
proged_data = data.drop(columns=['Tw', 'Ta', 'eta'])
proged_data.head()

,Q,theta,deltaT,one_over_eta
0,1.711929,0.321921,42.217191,2.864645
1,0.067135,2.705824,8.535838,1.217925
2,0.034759,1.873776,3.654556,1.322609
3,0.417783,2.089589,12.586385,1.937483
4,1.667614,0.328350,34.837551,12.000908


Uporabil bom gramatiko, ki dela linearne izraze v produktih(kvocientih) spremenljivk đ. Prioritiziral bom enačbe z malo plusi (sledim klasičnih oblikam fizikalnih formul), prioritiziral bom spremenljivko sin(theta).

In [3]:
# priprava naivne gramatike
# grammar = "E -> E '+' 'c' '*' V [0.1] | 'c' '*' V [0.9]\n"
grammar = "V -> V '*' F [0.6]| 'c' '*' F [0.4]\n" #  | V '/' F [0.2] 
grammar += "F -> 'theta' [0.1] | 'cos' '(' 'theta'  ')' [0.3] | 'deltaT' [0.3] | 'one_over_eta' [0.3]"

grammar = pg.GeneratorGrammar(grammar)

In [4]:
ED = pg.EqDisco(data=proged_data, 
                sample_size=500,
                lhs_vars=["Q"],
                rhs_vars=["theta", "deltaT", "1/eta"],
                strategy_settings = {"max_repeat":1000},
                generator = grammar,
                verbosity=1)

In [5]:
ED.generate_models()

[c*deltaT**2]
[c*deltaT]
[c*cos(theta)]
[c*theta**3]
[c*deltaT*cos(theta)]
[c*deltaT*cos(theta)]
[c*deltaT**2*theta]
[c*deltaT**2*theta]
[c*deltaT**2*theta]
[c*deltaT**2*theta]
[c*deltaT**2*cos(theta)**5]
[c*cos(theta)**2]
[c*theta]
[c*deltaT*one_over_eta]
[c*deltaT**4*one_over_eta*theta*cos(theta)**3]
[c*deltaT*one_over_eta*theta*cos(theta)]
[c*deltaT**2*one_over_eta**2*cos(theta)]
[c*deltaT**2*one_over_eta**2*cos(theta)]
[c*deltaT**2*one_over_eta**2*cos(theta)]
[c*deltaT**2*one_over_eta**2*cos(theta)]
[c*deltaT**2*one_over_eta**2*cos(theta)]
[c*deltaT**3*one_over_eta**3*cos(theta)]
[c*deltaT**3*one_over_eta**3*cos(theta)]
[c*deltaT**3*one_over_eta**3*cos(theta)]
[c*deltaT**3*one_over_eta**3*cos(theta)]
[c*deltaT*one_over_eta**3]
[c*deltaT*one_over_eta**3]
[c*one_over_eta**2*cos(theta)**2]
[c*one_over_eta**2*cos(theta)**2]
[c*one_over_eta**2*cos(theta)**2]
[c*one_over_eta**2*cos(theta)**2]
[c*deltaT*one_over_eta*cos(theta)]
[c*theta*cos(theta)**2]
[c*theta*cos(theta)**2]
[c*one_over_e

ModelBox: 500 models
-> [c*deltaT**2], p = 0.021599999999999998
-> [c*deltaT], p = 0.12
-> [c*cos(theta)], p = 0.12
-> [c*theta**3], p = 0.00014400000000000003
-> [c*deltaT*cos(theta)], p = 0.043199999999999995
-> [c*deltaT**2*theta], p = 0.003888
-> [c*deltaT**2*cos(theta)**5], p = 4.081466879999998e-06
-> [c*cos(theta)**2], p = 0.021599999999999998
-> [c*theta], p = 0.04000000000000001
-> [c*deltaT*one_over_eta], p = 0.043199999999999995
-> [c*deltaT**4*one_over_eta*theta*cos(theta)**3], p = 2.2039921151999992e-07
-> [c*deltaT*one_over_eta*theta*cos(theta)], p = 0.005598719999999996
-> [c*deltaT**2*one_over_eta**2*cos(theta)], p = 0.003275251199999999
-> [c*deltaT**3*one_over_eta**3*cos(theta)], p = 3.6733201919999976e-05
-> [c*deltaT*one_over_eta**3], p = 0.0027993599999999994
-> [c*one_over_eta**2*cos(theta)**2], p = 0.004199039999999999
-> [c*deltaT*one_over_eta*cos(theta)], p = 0.023327999999999995
-> [c*theta*cos(theta)**2], p = 0.003888
-> [c*one_over_eta*cos(theta)], p = 0.043

In [6]:
ED.fit_models()

ModelBox: 500 models
-> [0.00208863155644536*deltaT**2], p = 0.021599999999999998, error = 0.6556113356661527, time = 0.18965721130371094
-> [0.0563767583856052*deltaT], p = 0.12, error = 0.7560964680393287, time = 0.13784074783325195
-> [0.0161840936823956*cos(theta)], p = 0.12, error = 1.2713405897993268, time = 0.04746055603027344
-> [0.0306150350590362*theta**3], p = 0.00014400000000000003, error = 1.2226484279777616, time = 0.1310257911682129
-> [0.00240772519430417*deltaT*cos(theta)], p = 0.043199999999999995, error = 1.2710289378107122, time = 0.10943603515625
-> [0.00107152571936803*deltaT**2*theta], p = 0.003888, error = 0.8261948614369855, time = 0.2217569351196289
-> [0.000133668137037507*deltaT**2*cos(theta)**5], p = 4.081466879999998e-06, error = 1.2709217131383665, time = 0.18060040473937988
-> [0.571913943307724*cos(theta)**2], p = 0.021599999999999998, error = 1.2221660655705235, time = 0.0683138370513916
-> [0.337834861309631*theta], p = 0.04000000000000001, error = 1.

In [7]:
ED.get_results(5)

ModelBox: 5 models
-> [0.00208863884866988*deltaT**2], p = 7.346640383999997e-07, error = 0.6556113356546425, time = 0.3012824058532715
-> [0.0020886386685374*deltaT**2], p = 0.011663999999999997, error = 0.6556113356546611, time = 0.5064349174499512
-> [0.00208863989008618*deltaT**2], p = 0.0011337407999999997, error = 0.6556113356547991, time = 0.18055081367492676
-> [0.00208863640269478*deltaT**2], p = 0.004199039999999999, error = 0.6556113356560442, time = 0.25513458251953125
-> [0.00208863155644536*deltaT**2], p = 0.021599999999999998, error = 0.6556113356661527, time = 0.18965721130371094

In [8]:
def f_gramatika(df):
    return 0.00208864*df['deltaT']**2

def povprecna_kvadratna_napaka(f, data):
    return np.sum((f(data) -  data['Q'])**2)/len(data)

povprecna_kvadratna_napaka(f_gramatika, data)

0.4298262234391257

Gramatike se obnesejo slabo. Poskusil bom še z linearno regresijo. Ponovno bom dodal značilki 1/eta in sin(theta), ter deltaT. Zanimajo me linearni izrazi v produktih spremenljivk.

In [9]:
data['sin_of_theta'] = np.sin(data['theta'])
data_lr = data.drop(columns=['Tw', 'Ta', 'eta', 'theta'])

poly = PolynomialFeatures(5, include_bias=False)
X = poly.fit_transform(data_lr.drop('Q', axis=1))

imena_stolpcev = poly.get_feature_names_out()
data_lr = pd.DataFrame(X, columns=imena_stolpcev)
data_lr.head()

,deltaT,one_over_eta,sin_of_theta,deltaT^2,deltaT one_over_eta,deltaT sin_of_theta,one_over_eta^2,one_over_eta sin_of_theta,sin_of_theta^2,deltaT^3,...,deltaT one_over_eta^3 sin_of_theta,deltaT one_over_eta^2 sin_of_theta^2,deltaT one_over_eta sin_of_theta^3,deltaT sin_of_theta^4,one_over_eta^5,one_over_eta^4 sin_of_theta,one_over_eta^3 sin_of_theta^2,one_over_eta^2 sin_of_theta^3,one_over_eta sin_of_theta^4,sin_of_theta^5
0,42.217191,2.864645,0.316390,1782.291254,120.937275,13.357081,8.206192,0.906344,0.100102,75243.331122,...,313.995968,34.679708,3.830247,0.423037,192.909764,21.306179,2.353190,0.259901,0.028705,0.003170
1,8.535838,1.217925,0.422107,72.860533,10.396010,3.603040,1.483341,0.514095,0.178175,621.925724,...,6.509246,2.255969,0.781872,0.270980,2.679802,0.928763,0.321890,0.111560,0.038664,0.013400
2,3.654556,1.322609,0.954452,13.355782,4.833548,3.488097,1.749294,1.262366,0.910978,48.809456,...,8.070168,5.823782,4.202693,3.032845,4.047219,2.920648,2.107666,1.520983,1.097607,0.792081
3,12.586385,1.937483,0.868418,158.417075,24.385901,10.930248,3.753839,1.682546,0.754151,1993.898215,...,79.495662,35.631544,15.970769,7.158418,27.301659,12.237149,5.484935,2.458457,1.101930,0.493907
4,34.837551,12.000908,0.322482,1213.654987,418.082233,11.234482,144.021782,3.870076,0.103995,42280.768004,...,19417.589949,521.779089,14.020969,0.376764,248926.107778,6689.009194,179.743476,4.829971,0.129788,0.003488


In [10]:
from lr import lasso_regresija
izraz, napaka, beta = lasso_regresija(data_lr, data['Q'], lam=1)
print(izraz)
print(napaka)

500
0.412*deltaT + 0.623*one_over_eta + 0.173*sin_of_theta + 0.470*deltaT^2 + 0.316*deltaT one_over_eta + 0.015*deltaT sin_of_theta + 0.176*one_over_eta^2 + 0.871*one_over_eta sin_of_theta + 0.475*sin_of_theta^2 + 0.160*deltaT^3 + 0.915*deltaT^2 one_over_eta + 0.323*deltaT^2 sin_of_theta + 0.770*deltaT one_over_eta^2 + 0.050*deltaT one_over_eta sin_of_theta + 0.644*deltaT sin_of_theta^2 + 0.239*one_over_eta^3 + 0.357*one_over_eta^2 sin_of_theta + 0.207*one_over_eta sin_of_theta^2 + 0.329*sin_of_theta^3 + 0.066*deltaT^4 + 0.332*deltaT^3 one_over_eta + 0.021*deltaT^3 sin_of_theta + 0.217*deltaT^2 one_over_eta^2 + 0.388*deltaT^2 one_over_eta sin_of_theta + 0.455*deltaT^2 sin_of_theta^2 + 0.129*deltaT one_over_eta^3 + 0.259*deltaT one_over_eta^2 sin_of_theta + 0.459*deltaT one_over_eta sin_of_theta^2 + 0.719*deltaT sin_of_theta^3 + 0.126*one_over_eta^4 + 0.343*one_over_eta^3 sin_of_theta + 0.624*one_over_eta^2 sin_of_theta^2 + 0.798*one_over_eta sin_of_theta^3 + 0.245*sin_of_theta^4 + 0.05

In [11]:
def f_lr(data):
    data['sin_of_theta'] = np.sin(data['theta'])
    data_lr = data.drop(columns=['Tw', 'Ta', 'eta', 'theta'])
    poly = PolynomialFeatures(5, include_bias=False)
    X = poly.fit_transform(data_lr.drop('Q', axis=1))
    imena_stolpcev = poly.get_feature_names_out()
    data_lr = pd.DataFrame(X, columns=imena_stolpcev)
    # izračun Q
    return data_lr.dot(beta)

povprecna_kvadratna_napaka(f_lr, data)

71820990853669.36

Tudi linearna regresija ne daje odličnih rezultatov. 

In [12]:
data_pysr = data[[ 'deltaT', 'sin_of_theta', 'one_over_eta']]
data_pysr.head()

,deltaT,sin_of_theta,one_over_eta
0,42.217191,0.316390,2.864645
1,8.535838,0.422107,1.217925
2,3.654556,0.954452,1.322609
3,12.586385,0.868418,1.937483
4,34.837551,0.322482,12.000908


In [13]:
from pysr import PySRRegressor

model = PySRRegressor(
    niterations=100,  # < Increase me for better results
    binary_operators=["+", "*"],
    unary_operators=[
        # "cos",
        # "exp",
        # "sin",
        "inv(x) = 1/x",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)

In [14]:
model.fit(data_pysr, data['Q'])

/home/urh/.local/lib/python3.10/site-packages/pysr/sr.py:1296: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/urh/.local/lib/python3.10/site-packages/pysr/sr.py:1364: FutureWarning: Index.is_object is deprecated.Use pandas.api.types.is_object_dtype instead
  if X.columns.is_object() and X.columns.str.contains(" ").any():


Compiling Julia backend...


/home/urh/.local/lib/python3.10/site-packages/julia/juliainfo.py:93: UserWarning: julia warned:
Juliaup configuration is locked by another process, waiting for it to unlock.
  warnings.warn("{} warned:\n{}".format(julia, stderr))
/home/urh/.local/lib/python3.10/site-packages/julia/juliainfo.py:93: UserWarning: julia warned:
Juliaup configuration is locked by another process, waiting for it to unlock.
  warnings.warn("{} warned:\n{}".format(julia, stderr))
/home/urh/.local/lib/python3.10/site-packages/pysr/julia_helpers.py:195: UserWarning: Your system's Python library is static (e.g., conda), so precompilation will be turned off. For a dynamic library, try `pyenv`.
  warnings.warn(


Started!

Expressions evaluated per second: 7.290e+04
Head worker occupation: 13.5%
Progress: 166 / 1500 total iterations (11.067%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.026e+00  1.144e-01  sin_of_theta
3           5.717e-01  2.922e-01  (0.056376774 * deltaT)
5           3.159e-01  2.965e-01  ((sin_of_theta * 0.08760614) * deltaT)
7           1.251e-01  4.632e-01  ((0.0032668805 * (deltaT * deltaT)) * sin_of_theta)
9           1.226e-01  1.009e-02  ((0.0032617897 * ((deltaT * deltaT) + one_over_eta)) * sin_of_...
                                  theta)
11          1.114e-01  4.778e-02  (0.0031340758 * ((deltaT + (one_over_eta * 0.22086121)) * (del...
                                  taT * sin_of_theta)))
12          1.001e-01  1.076e-01  ((0.0032617897 * deltaT) * ((deltaT + inv(-0.19932508 * one_ov...
                                  er_eta)) * sin_of_

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                       sin_of_theta   
	1         0.292234                             (0.056376774 * deltaT)   
	2         0.296516             ((sin_of_theta * 0.08760614) * deltaT)   
	3         0.463177  ((0.0032668805 * (deltaT * deltaT)) * sin_of_t...   
	4         0.010104  ((0.0032606693 * ((deltaT * deltaT) + one_over...   
	5         0.047874  (0.0031340758 * (((0.2255092 * one_over_eta) +...   
	6         0.457375  (((0.0035923906 * (deltaT + inv(one_over_eta *...   
	7         0.195868  ((((-0.00462467 * (deltaT + inv(-0.06772108 * ...   
	8   >>>>  0.637115  ((0.0035923906 * deltaT) * (((1.3285637 * delt...   
	9         0.004498  ((0.0035923906 * deltaT) * (((1.3285637 * delt...   
	10        0.000209  ((0.0035923906 * (deltaT + 0.09100331)) * (((1...   
	
	        loss  complexity  
	0   1.025619           1  
	1   0.571682           3  
	2   0.315940           5  
	3   0.125111           7  
	4   0.122608           9  
	5   0.111413          11  
	6   0.070518          12  
	7   0.047662          14  
	8   0.013328          16  
	9   0.013209          18  
	10  0.013204          20  
]

In [15]:
model.get_best()
print(model.get_best()['sympy_format'])

0.0035923906*deltaT*sin_of_theta*(1.3285637*deltaT - 0.883914610312985*deltaT/one_over_eta)


In [20]:
# 0.005024702*deltaT**2*sin_of_theta*(0.949868 - 0.63198835/one_over_eta)
def f_pysr(data):
    return 0.005024702*data['deltaT']**2*data['sin_of_theta']*(0.949868 - 0.63198835/data['one_over_eta'])
povprecna_kvadratna_napaka(f_pysr, data)

0.013328424604298457

## Ročna oblika
Poskusil bom še z ročno hervistiko. Iskal bom enačbo oblike deltaT^csin(theta)^c eta^c

In [16]:
data_rocno = data[ ['deltaT', 'sin_of_theta', 'one_over_eta']]
data_rocno.head()

,deltaT,sin_of_theta,one_over_eta
0,42.217191,0.316390,2.864645
1,8.535838,0.422107,1.217925
2,3.654556,0.954452,1.322609
3,12.586385,0.868418,1.937483
4,34.837551,0.322482,12.000908


In [17]:
from scipy.optimize import minimize

def equation(params):
    a, b, c, d = params
    q = a * data['deltaT']**b * data['sin_of_theta']**c * data['one_over_eta']**d
    return np.sum((data['Q'] - q)**2) / len(data)

results= minimize(equation, [1,1,1,1])
results["x"]

array([0.0021297 , 2.05180646, 1.02670162, 0.22636962])

In [18]:
a, b, c, d = results["x"]
print(f'Povprečna kvadratna napaka enačbe {a}deltaT^{b}sin(theta)^{c}eta^(-{d}) je  {results["fun"]}' )


Povprečna kvadratna napaka enačbe 0.0021297039865321972deltaT^2.0518064574801134sin(theta)^1.0267016242714626eta^(-0.2263696173867843) je  0.0375624885916007
